In [1]:
import ee
import geemap

# Initialize
try:
    ee.Initialize(project='golf-mapper-463720')
    print("Earth Engine initialized successfully.")
except Exception as e:
    print(f"Initialization failed: {e}")
    ee.Authenticate()
    ee.Initialize(project='golf-mapper-463720')

# Better authentication test
try:
    # Test with a simple operation that requires auth
    point = ee.Geometry.Point([-122.0, 37.0])
    image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_044034_20140318')
    print("✅ Successfully authenticated - can access Earth Engine")
except Exception as e:
    print(f"❌ Authentication failed: {e}")

/Users/carrollcj/proj/golfmapper/venv/lib/python3.11/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Earth Engine initialized successfully.
✅ Successfully authenticated - can access Earth Engine


In [ ]:
import ee
import geemap
import geopandas as gpd
import os
from shapely.ops import transform


# Earth Engine Initialization & Authentication
print("Initializing Google Earth Engine...")
try:
    ee.Initialize(project='golf-mapper-463720')
    print("Earth Engine initialized successfully.")
except Exception as e:
    print("Earth Engine not initialized. Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project='golf-mapper-463720')
    print("Earth Engine re-authenticated and initialized.")


# Authentication test
try:
    # Test with a simple operation that requires auth
    point = ee.Geometry.Point([-122.0, 37.0])
    image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_044034_20140318')
    print("✅ Successfully authenticated - can access Earth Engine")
except Exception as e:
    print(f"❌ Authentication failed: {e}")


# try:
#     account = ee.data.getAssetRoots()[0]['id']
#     print(f"✅ Authenticated as: {account}")
# except:
#     print("❌ Still not authenticated")

Initializing Google Earth Engine...
Earth Engine initialized successfully.
✅ Successfully authenticated - can access Earth Engine




earthengine can only deal with (x,y) coordinates, and earth pro exports KML files with a Z coordinate. The code below removes the z coordinate. FIXME ADD TRY/CATCH BLOCK

<h1>cleaning the data test</h1>

In [ ]:
# Read your KML

x = gpd.read_file('/Users/carrollcj/proj/golfmapper/kml_files/test3.kml')

# Function to remove Z coordinates
def remove_z_dimension(geom):
    if geom.has_z:
        return transform(lambda x, y, z=None: (x, y), geom)
    return geom

# Apply to all geometries
x['geometry'] = x.geometry.apply(remove_z_dimension)

# Verify it worked
print("After Z removal:")
print("Geometry types:", x.geometry.type.value_counts())
print("Sample geometry:", x.geometry.iloc[0])

# Now try the Earth Engine conversion
#gdf_to_ee turns geopandas dataframes into earthengine objects

geometry = geemap.gdf_to_ee(x)
print("Success! Converted to Earth Engine geometry")
print('===============')

ge = geometry.geometry()
print(f"FIXED GEOMETRY TYPE: {type(ge)}")
print(f"FIXED GEOMETRY INFO: {ge.getInfo()}")
# print('ge type:', ge.type())
print('===============')

#clipping the image
start_date = '2023-01-01'
end_date = '2023-12-31'

# Get Sentinel-2 imagery
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(ge) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()  # Get median composite

# Clip the image to your polygon
clipped_image = sentinel.clip(ge)

print('===============')
print('EXPORTING THE IMAGE')

task = ee.batch.Export.image.toDrive(
    image=clipped_image,
    description='clipped_image_test',
    folder='GEE_exports',  # Optional: creates folder in Drive
    fileNamePrefix='test1',
    scale=30,  # Adjust resolution
    region=geo_fixed,
    maxPixels=1e9
)
task.start()

print(f"Task status: {task.status()}")

# Check if geometry variable exists and is valid
print(f"Geometry exists: {ge is not None}")
print(f"Geometry type: {type(ge)}")
print(f"Geometry info: {ge.getInfo()}")

# # Debug your gdf_to_ee conversion
# print(f"GDF shape: {gdf.shape}")
# print(f"GDF CRS: {gdf.crs}")
# print(f"GDF bounds: {gdf.bounds}")

# # Make sure conversion worked
# geometry = geemap.gdf_to_ee(gdf)
# print(f"Converted geometry: {geometry.getInfo()}")


After Z removal:
Geometry types: Polygon    1
Name: count, dtype: int64
Sample geometry: POLYGON ((-95.37972929310104 37.65883757523466, -95.35579120123668 37.65832987171771, -95.35416643387549 37.66965512912511, -95.3815263901972 37.66997271533956, -95.37972929310104 37.65883757523466))
Success! Converted to Earth Engine geometry
FIXED GEOMETRY TYPE: <class 'ee.geometry.Geometry'>
FIXED GEOMETRY INFO: {'type': 'Polygon', 'coordinates': [[[-95.37972929310104, 37.65883757523466], [-95.35579120123668, 37.65832987171771], [-95.35416643387549, 37.66965512912511], [-95.3815263901972, 37.66997271533956], [-95.37972929310104, 37.65883757523466]]]}
EXPORTING THE IMAGE
Task status: {'state': 'READY', 'description': 'clipped_image_test', 'priority': 100, 'creation_timestamp_ms': 1752419623432, 'update_timestamp_ms': 1752419623432, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'SOU4BJZW4VCPACUQ6EX27BH3', 'name': 'projects/golf-mapper-463720/operations/SOU4BJZW4VCPACUQ6EX27BH3'}
Geom

<h1>cleaning the data functional</h1>

In [ ]:
# Read your KML
#gdf = geopandas data frame
#gdf_to_ee() is a built-in earthengine function
gdf = gpd.read_file('/Users/carrollcj/proj/golfmapper/kml_files/test3.kml')

# Function to remove Z coordinates
def remove_z_dimension(geom):
    if geom.has_z:
        return transform(lambda x, y, z=None: (x, y), geom)
    return geom

# Apply to all geometries
gdf['geometry'] = gdf.geometry.apply(remove_z_dimension)

# Verify it worked
print("After Z removal:")
print("Geometry types:", gdf.geometry.type.value_counts())
print("Sample geometry:", gdf.geometry.iloc[0])

# Now try the Earth Engine conversion
geometry = geemap.gdf_to_ee(gdf)
print("Success! Converted to Earth Engine geometry")

###########
# print("Geometry types:", gdf.geometry.type.value_counts())


# print('===')
# print('fixing the geometry type:')
# geo_fixed = geometry.geometry()
# geo_test = geemap.gdf_to_ee(gdf)

# print("Geometry types:", gdf.geometry.type.value_counts())

##########


After Z removal:
Geometry types: Polygon    1
Name: count, dtype: int64
Sample geometry: POLYGON ((-95.37972929310104 37.65883757523466, -95.35579120123668 37.65832987171771, -95.35416643387549 37.66965512912511, -95.3815263901972 37.66997271533956, -95.37972929310104 37.65883757523466))
Success! Converted to Earth Engine geometry
Geometry types: Polygon    1
Name: count, dtype: int64
===
fixing the geometry type:
Geometry types: Polygon    1
Name: count, dtype: int64


<h1>clipping the satellite image</h1>

In [ ]:
start_date = '2023-01-01'
end_date = '2023-12-31'

# Get Sentinel-2 imagery
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()  # Get median composite

# Clip the image to your polygon
clipped_image = sentinel.clip(geometry)






<h1>exporting the iamge to drive</h1>

In [ ]:
#TESTING THE FIXED GEOMETRY
#when the geomtery from cell 2 was added, it wasnt in the 'Geometry' format
#EE needs this data type to export the image
#the code below (line 7-38) gets stuck in a 'READY' state when its run

##################################################
# geo_fixed = geometry.geometry()


# print(f"FIXED GEOMETRY TYPE: {type(geo_fixed)}")
# print(f"FIXED GEOMETRY INFO: {geo_fixed.getInfo()}")

# task = ee.batch.Export.image.toDrive(
#     image=clipped_image,
#     description='clipped_image_test',
#     folder='GEE_exports',  # Optional: creates folder in Drive
#     fileNamePrefix='test1',
#     scale=30,  # Adjust resolution
#     region=geo_fixed,
#     maxPixels=1e9
# )
# task.start()

# print(f"Task status: {task.status()}")

# # Check if geometry variable exists and is valid
# print(f"Geometry exists: {geometry is not None}")
# print(f"Geometry type: {type(geometry)}")
# print(f"Geometry info: {geometry.getInfo()}")

# # Debug your gdf_to_ee conversion
# print(f"GDF shape: {gdf.shape}")
# print(f"GDF CRS: {gdf.crs}")
# print(f"GDF bounds: {gdf.bounds}")

# # Make sure conversion worked
# geometry = geemap.gdf_to_ee(gdf)
# print(f"Converted geometry: {geometry.getInfo()}")
################################################################

#trying a simpler test


# Cancel the stuck task first
# task.cancel()

#the code below (49-78) has the same issue (simpler image)
########################################################
# geo_fixed = geometry.geometry()


# print(f"FIXED GEOMETRY TYPE: {type(geo_fixed)}")
# print(f"FIXED GEOMETRY INFO: {geo_fixed.getInfo()}")


# # Try exporting just a single band with coarser resolution
# simple_task = ee.batch.Export.image.toDrive(
#     image=clipped_image.select('B4'),  # Just one band
#     description='simple_single_band',
#     folder='GEE_exports',
#     fileNamePrefix='simple_b4',
#     scale=100,  # Coarser resolution
#     region=geo_fixed,
#     maxPixels=1e7,  # Much smaller max pixels
#     fileFormat='GeoTIFF'
# )
# simple_task.start()

# # Check this task's status
# print(f"Simple task status: {simple_task.status()}")

# # See if you have other tasks that might be blocking
# tasks = ee.batch.Task.list()
# ready_tasks = [t for t in tasks if t.state == 'READY']
# running_tasks = [t for t in tasks if t.state == 'RUNNING']

# print(f"Ready tasks: {len(ready_tasks)}")
# print(f"Running tasks: {len(running_tasks)}")

###############################################################

#importing an stock image that SHOULD work

geo_fixed = geometry.geometry()


print(f"FIXED GEOMETRY TYPE: {type(geo_fixed)}")
print(f"FIXED GEOMETRY INFO: {geo_fixed.getInfo()}")



# Test if the clipped image has any pixels
pixel_count = clipped_image.select('B4').reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=geo_fixed,
    scale=100,
    maxPixels=1e7
).getInfo()
print(f"Pixel count in region: {pixel_count}")

# Check if the original collection found any images
collection_size = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(geo_fixed) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .size()
print(f"Images found in collection: {collection_size.getInfo()}")


# Cancel your current task first
# simple_task.cancel()

# Let's find a Landsat image that definitely exists in your area
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(geo_fixed) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .first()

print(f"Found Landsat image: {landsat_collection.get('system:id').getInfo()}")

# Export this image
test_task = ee.batch.Export.image.toDrive(
    image=landsat_collection.select('SR_B4'),
    description='landsat_test_v2',
    folder='GEE_exports',
    fileNamePrefix='landsat_test_v2',
    scale=100,
    region=geo_fixed,
    maxPixels=1e7,
    fileFormat='GeoTIFF'
)
test_task.start()
print(f"Landsat test task status: {test_task.status()}")




FIXED GEOMETRY TYPE: <class 'ee.geometry.Geometry'>
FIXED GEOMETRY INFO: {'type': 'Polygon', 'coordinates': [[[-95.37972929310104, 37.65883757523466], [-95.35579120123668, 37.65832987171771], [-95.35416643387549, 37.66965512912511], [-95.3815263901972, 37.66997271533956], [-95.37972929310104, 37.65883757523466]]]}
Pixel count in region: {'B4': 364}
Images found in collection: 50
Found Landsat image: LANDSAT/LC08/C02/T1_L2/LC08_026034_20230329
Landsat test task status: {'state': 'READY', 'description': 'landsat_test_v2', 'priority': 100, 'creation_timestamp_ms': 1752371738225, 'update_timestamp_ms': 1752371738225, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '2ZCUBMUMQNVWYI2OMLMZBXK4', 'name': 'projects/golf-mapper-463720/operations/2ZCUBMUMQNVWYI2OMLMZBXK4'}


FINAL TEST

In [19]:
# Try with elevation data (always available)
elevation = ee.Image('USGS/SRTMGL1_003')
elev_task = ee.batch.Export.image.toDrive(
    image=elevation,
    description='elevation_test',
    folder='GEE_exports',
    fileNamePrefix='elevation_test',
    scale=100,
    region=geo_fixed,
    maxPixels=1e7,
    fileFormat='GeoTIFF'
)
elev_task.start()
print(f"Elevation test task status: {elev_task.status()}")

# Check if you can access basic Earth Engine data - CORRECTED
try:
    # Option 1: Get info about the image using getInfo() on the ee.Image object
    project_info = elevation.getInfo()
    print("✅ Can access basic Earth Engine data")
    
    # Option 2: Or get asset info using the asset ID string
    # asset_info = ee.data.getInfo('USGS/SRTMGL1_003')
    # print("✅ Can access asset info")
    
except Exception as e:
    print(f"❌ Data access issue: {e}")

# Try getting project assets
try:
    assets = ee.data.getList({'id': 'projects/golf-mapper-463720/assets'})
    print(f"✅ Can access project assets: {len(assets)}")
except Exception as e:
    print(f"❌ Project access issue: {e}")

# Additional debugging - check task more thoroughly
import time
print("\nWaiting a few seconds for task to process...")
time.sleep(3)
print(f"Task status after wait: {elev_task.status()}")

# Get more detailed task info
task_info = elev_task.status()
if 'error_message' in task_info:
    print(f"❌ Task error: {task_info['error_message']}")
else:
    print(f"Task state: {task_info.get('state', 'Unknown')}")
    if task_info.get('state') == 'COMPLETED':
        print("✅ Export completed successfully!")
    elif task_info.get('state') == 'FAILED':
        print("❌ Export failed")
    elif task_info.get('state') == 'RUNNING':
        print("⏳ Export is still running...")

Elevation test task status: {'state': 'READY', 'description': 'elevation_test', 'priority': 100, 'creation_timestamp_ms': 1752372118244, 'update_timestamp_ms': 1752372118244, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'ZST7XOE5DBRFW44PLF7QHRUJ', 'name': 'projects/golf-mapper-463720/operations/ZST7XOE5DBRFW44PLF7QHRUJ'}
✅ Can access basic Earth Engine data
✅ Can access project assets: 0

Waiting a few seconds for task to process...
Task status after wait: {'state': 'READY', 'description': 'elevation_test', 'priority': 100, 'creation_timestamp_ms': 1752372118244, 'update_timestamp_ms': 1752372118244, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'ZST7XOE5DBRFW44PLF7QHRUJ', 'name': 'projects/golf-mapper-463720/operations/ZST7XOE5DBRFW44PLF7QHRUJ'}
Task state: READY


In [ ]:
# import ee

# # Force re-authentication
# ee.Authenticate()
# ee.Initialize()

# # Then verify it worked
# try:
#     account = ee.data.getAssetRoots()[0]['id']
#     print(f"✅ Authenticated as: {account}")
# except:
#     print("❌ Still not authenticated")

    
start_date = '2023-01-01'
end_date = '2023-12-31'

# Get Sentinel-2 imagery
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()  # Get median composite

# Clip the image to your polygon
clipped_image = sentinel.clip(geometry)


# For raster/image export
task = ee.batch.Export.image.toDrive(
    image=clipped_image,
    description='clipped_image_test',
    folder='GEE_exports',  # Optional: creates folder in Drive
    fileNamePrefix='test1',
    scale=30,  # Adjust resolution
    region=geometry,
    maxPixels=1e9
)
task.start()


import time

while task.status()['state'] in ['READY', 'RUNNING']:
    print(f"Task status: {task.status()['state']}")
    time.sleep(10)
print(f"Task completed: {task.status()['state']}")



# # More robust export
# try:
#     # Check if image has any data
#     pixel_count = clipped_image.select('B4').reduceRegion(
#         reducer=ee.Reducer.count(),
#         geometry=geometry,
#         scale=100,  # Coarser scale for counting
#         maxPixels=1e6
#     ).getInfo()
#     print("Pixel count:", pixel_count)
    
#     if pixel_count['B4'] > 0:
#         task = ee.batch.Export.image.toDrive(
#             image=clipped_image,
#             description='sentinel2_2023_clipped_v2',
#             folder='GEE_exports',
#             fileNamePrefix='sentinel2_2023_median_v2',
#             region=geometry,
#             scale=10,
#             crs='EPSG:4326',
#             maxPixels=1e9,
#             fileFormat='GeoTIFF'
#         )
#         task.start()
#         print(f"Task started: {task.id}")
#     else:
#         print("No valid pixels found in the region")
        
# except Exception as e:
#     print(f"Error: {e}")


# # Check your Google account info
# print("GEE Account:", ee.data.getAssetRoots()[0]['id'] if ee.data.getAssetRoots() else "No account found")

# # Try a very simple export with explicit parameters
# simple_task = ee.batch.Export.image.toDrive(
#     image=clipped_image.select('B4'),  # Just one band
#     description='simple_test',
#     fileNamePrefix='simple_test',
#     region=geometry,
#     scale=30,
#     maxPixels=1e6
# )
# simple_task.start()
# print(f"Simple task started: {simple_task.id}")
# # print(ee.data.getAssetRoots())


# # Try getting more detailed task info
# tasks = ee.batch.Task.list()
# for task in tasks[:5]:
#     if task.state == 'READY':
#         print(f"✅ Task ID: {task.id}")
#         print(f"Description: {task.config.get('description', 'N/A')}")
#         print(f"State: {task.state}")
#         print(f"Task type: {task.task_type}")
#         print(f"All available attributes: {dir(task)}")
#         print("=" * 50)




In [ ]:
start_date = '2023-01-01'
end_date = '2023-12-31'

# Get Sentinel-2 imagery
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()  # Get median composite

# Clip the image to your polygon
clipped_image = sentinel.clip(geometry)


# More robust export
try:
    # Check if image has any data
    pixel_count = clipped_image.select('B4').reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=geometry,
        scale=100,  # Coarser scale for counting
        maxPixels=1e6
    ).getInfo()
    print("Pixel count:", pixel_count)
    
    if pixel_count['B4'] > 0:
        task = ee.batch.Export.image.toDrive(
            image=clipped_image,
            description='sentinel2_2023_clipped_v2',
            folder='GEE_exports',
            fileNamePrefix='sentinel2_2023_median_v2',
            region=geometry,
            scale=10,
            crs='EPSG:4326',
            maxPixels=1e9,
            fileFormat='GeoTIFF'
        )
        task.start()
        print(f"Task started: {task.id}")
    else:
        print("No valid pixels found in the region")
        
except Exception as e:
    print(f"Error: {e}")













# Download directly to local machine
# geemap.ee_export_image_to_drive(
#     clipped_image,
#     description='my_clipped_image',
#     folder='/Users/carrollcj/proj/golfmapper/exported_images',
#     region=geometry,
#     scale=10,
#     crs='EPSG:4326'
# )

# Export to Google Drive
task = ee.batch.Export.image.toDrive(
    image=clipped_image,
    description='sentinel2_2023_clipped',
    folder='GEE_exports',  # folder in your Drive
    fileNamePrefix='sentinel2_2023_median',
    region=geometry,
    scale=10,  # Sentinel-2 resolution
    crs='EPSG:4326',
    maxPixels=1e9,
    fileFormat='GeoTIFF'
)
task.start()

# Check if your task is still running
# tasks = ee.batch.Task.list()
# for task in tasks[:5]:  # Show last 5 tasks
#     print(f"Task ID: {task.id}")
#     print(f"Description: {task.config['description']}")
#     print(f"Status: {task.state}")
#     # print(f"Creation time: {task.creation_timestamp_ms}")
#     print("---")

# Get detailed export info
tasks = ee.batch.Task.list()
for task in tasks[:5]:
    if task.state == 'READY':
        print(f"✅ Task ID: {task.id}")
        print(f"Description: {task.config['description']}")
        print(f"Full config: {task.config}")
        print("=" * 50)

print(ee.data.getAssetRoots())


#print('Export task started. Check your Google Drive.')



#FUNCTIONAL CODE TO VISUALIZE A GEOMETRY ON THE MAP

In [ ]:
#FIXME for additional polygons its best to use geopandas.
#Need to integrate, but for now keep the interactive map

# Create an Interactive Map
Map = geemap.Map()
print("Interactive map object created.")

print("geemap version:", geemap.__version__)

display(Map)

# adds the kml file onto the map
Map.add_kml('/Users/carrollcj/proj/golfmapper/kml_files/test2.kml')

In [ ]:
geometry = geemap.kml_to_ee('/Users/carrollcj/proj/golfmapper/kml_files/test2.kml')


start_date = '2023-01-01'
end_date = '2023-12-31'

# Get Sentinel-2 imagery
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()  # Get median composite

# Clip the image to your polygon
clipped_image = sentinel.clip(geometry)

# # Clip Sentinel imagery
# clipped_image = sentinel.clip(geometry)

In [ ]:
#checking to see the data shape


# gdf = gpd.read_file('/Users/carrollcj/proj/golfmapper/kml_files/test2.kml')

# gemoetry = geemap.gdf_to_ee(gdf)


import geopandas as gpd

gdf = gpd.read_file('/Users/carrollcj/proj/golfmapper/kml_files/test2.kml')

# Check the data structure
print("Shape:", gdf.shape)
print("Columns:", gdf.columns.tolist())
print("Geometry types:", gdf.geometry.type.value_counts())
print("CRS:", gdf.crs)
print("Sample data:")
print(gdf.head())

# Check for geometry issues
print("Null geometries:", gdf.geometry.isnull().sum())
print("Invalid geometries:", (~gdf.geometry.is_valid).sum())